## Obtaining data for public transport in London

In [ ]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_843c80ec5aab45029a5e149891fbb8df = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='YUtAXUI9oEDONulhvYIEX6FG2Vk0e_ncU8_3ujhSZjKX',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_843c80ec5aab45029a5e149891fbb8df.get_object(Bucket='coursera-donotdelete-pr-xuctsznipu8kkf',Key='Book2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_lon = pd.read_csv(body)
df_lon.head()


In [2]:
df_lon.dropna(inplace=True)
df_lon.drop(['FID','OBJECTID','EASTING','NORTHING','LINES','NETWORK','Zone'],axis=1,inplace=True)

In [3]:
df_lon.rename(columns={'x':'Longitude','y':'Latitude'},inplace=True)

### Searching for venues near the London public transport access points

In [21]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
LIMIT = 100

In [24]:
CLIENT_ID='removed'
CLIENT_SECRET='removed'
VERSION=20180605

In [25]:
df_venues = getNearbyVenues(names = df_lon['NAME'],latitudes = df_lon['Latitude'],longitudes = df_lon['Longitude'])

Temple
Blackfriars
Mansion House
Cannon Street
Monument
Tower Hill
Aldgate
Liverpool Street
Moorgate
Barbican
Farringdon
King's Cross St. Pancras
Euston Square
Great Portland Street
Baker Street
Edgware Road (Circle Line)
Paddington
Bayswater
High Street Kensington
Gloucester Road
South Kensington
Victoria
Pimlico
Warren Street
Queensway
Hyde Park Corner
Knightsbridge
Leicester Square
Covent Garden
Russell Square
Earl's Court
Notting Hill Gate
Lancaster Gate
Marble Arch
Bond Street
Bank
Oxford Circus
Holborn
Chancery Lane
St. Paul's
Paddington
Embankment
Westminster
Euston
Waterloo
Green Park
Paddington (H&C Line)
Piccadilly Circus
Charing Cross
Lambeth North
Edgware Road (Bakerloo)
Marylebone
Regent's Park
Sloane Square
St. James's Park
Goodge Street
Tottenham Court Road
Borough
Old Street
Angel
Aldgate East
Southwark
London Bridge
East Putney
Putney Bridge
Parsons Green
White City
Shepherd's Bush
Holland Park
Brixton
Vauxhall
Highbury & Islington
Caledonian Road
Holloway Road
Arsenal

### Removing venues other than pubs from the dataset

In [71]:
df_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Pubs
0,Temple,51.510474,-0.112644,Two Temple Place,51.511523,-0.112236,History Museum,0
1,Temple,51.510474,-0.112644,The Southbank Observation Point,51.508297,-0.111180,Scenic Lookout,0
2,Temple,51.510474,-0.112644,The Queen's Walk,51.508308,-0.110853,Scenic Lookout,0
3,Temple,51.510474,-0.112644,Temple Brew House,51.512940,-0.113029,Pub,1
4,Temple,51.510474,-0.112644,Temple Gardens,51.511154,-0.111472,Park,0


In [72]:
df_venues['Pubs'] = [1 if x =='Pub' else 0 for x in df_venues['Venue Category']] 

In [73]:
df_pubs = df_venues[df_venues['Pubs'] == 1]

In [77]:
df_pubs.reset_index()

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Pubs
0,3,Temple,51.510474,-0.112644,Temple Brew House,51.512940,-0.113029,Pub,1
1,6,Temple,51.510474,-0.112644,The Edgar Wallace,51.512585,-0.112819,Pub,1
2,8,Temple,51.510474,-0.112644,The Understudy,51.507725,-0.113724,Pub,1
3,30,Temple,51.510474,-0.112644,The Cheshire Cheese,51.512531,-0.113178,Pub,1
4,36,Temple,51.510474,-0.112644,The Old Bank of England,51.513870,-0.111491,Pub,1
5,43,Temple,51.510474,-0.112644,Savoy Tap,51.510889,-0.119509,Pub,1
6,52,Blackfriars,51.511114,-0.102020,Founder's Arms,51.508563,-0.101333,Pub,1
7,81,Blackfriars,51.511114,-0.102020,The Old Bell Tavern,51.514068,-0.105233,Pub,1
8,94,Blackfriars,51.511114,-0.102020,The Harrow,51.513004,-0.106752,Pub,1
9,193,Cannon Street,51.510963,-0.088801,The Counting House,51.513324,-0.085110,Pub,1


In [79]:
df_pubs.drop(['Neighborhood Latitude','Neighborhood Longitude','Pubs'],axis=1,inplace=True)

In [93]:
df_pubs.reset_index(inplace=True)

### Creating a map with all pubs obtained from Foursquare

In [87]:
!conda install -c conda-forge folium=0.5.0
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [114]:
map_london = folium.Map(location=[51.51,-0.11], zoom_start=12)

# add markers to map
for lat, lng, neighborhood, venue in zip(df_pubs['Venue Latitude'], df_pubs['Venue Longitude'], df_pubs['Neighborhood'], df_pubs['Venue']):
    label = '{}, {}'.format(venue,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

### Adding the data for public transport to the map

In [115]:

body = client_843c80ec5aab45029a5e149891fbb8df.get_object(Bucket='coursera-donotdelete-pr-xuctsznipu8kkf',Key='Book2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data = pd.read_csv(body)
df_data.head(11)


,FID,OBJECTID,NAME,EASTING,NORTHING,LINES,NETWORK,Zone,x,y
0,0,78,Temple,530959,180803,"District, Circle",London Underground,1,-0.112644,51.510474
1,1,79,Blackfriars,531694,180893,"District, Circle",London Underground,1,-0.102020,51.511114
2,2,80,Mansion House,532354,180932,"District, Circle",London Underground,1,-0.092495,51.511306
3,3,81,Cannon Street,532611,180900,"District, Circle",London Underground,1,-0.088801,51.510963
4,4,82,Monument,532912,180824,"District, Circle",London Underground,1,-0.084502,51.510209
5,5,83,Tower Hill,533579,180755,"District, Circle",London Underground,1,-0.074914,51.509434
6,6,84,Aldgate,533613,181262,"Metropolitan, Circle",London Underground,1,-0.074236,51.513982
7,7,85,Liverpool Street,533094,181567,"Metropolitan, Central, Circle, Hammersmith & City",London Underground,1,-0.081601,51.516842
8,8,86,Moorgate,532668,181668,"Metropolitan, Northern, Circle, Hammersmith & ...",London Underground,1,-0.087700,51.517853
9,9,87,Barbican,532004,181856,"Metropolitan, Circle, Hammersmith & City",London Underground,1,-0.097190,51.519699


In [116]:
for lat, lng, network, name in zip(df_data['y'], df_data['x'], df_data['NETWORK'], df_data['NAME']):
    label = '{}, {}'.format(name,network)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        parse_html=False).add_to(map_london) 
map_london

## From the map it can be concluded that Mayfair, Chelsea and Hackney are possible areas where a pub can be opened.